# Global snowmelt runoff onset processing

This notebook implements the core processing pipeline for detecting snowmelt runoff onset timing from Sentinel-1 SAR data at a global scale. The methodology detects the timing of minimum backscatter values, which correspond to snowmelt runoff onset.


## Processing pipeline
1. **Data acquisition**: Acquire Sentinel-1 RTC data from Microsoft Planetary Computer
1. **Snow masking**: Apply spatiotemporal snow cover constraints  
1. **Quality filtering**: Remove scenes with insufficient temporal sampling
1. **Calculate temporal resolution**: Calculate temporal resolution on the filtered dataset
1. **Runoff detection**: Calculate minimum backscatter timing per orbit
1. **Aggregate statistics**: Compute 10-year median and MAD for runoff onset, 10-year median for temporal resolution
1. **Output generation**: Write results to global zarr store

## Last checks!!

Before large-scale processing...

1. use [egagli/MODIS_seasonal_snow_mask](https://github.com/egagli/MODIS_seasonal_snow_mask) to create the seasonal snow cover dataset (contains: snow appearance date, snow disappearnce date, max consecutive number of snow cover days, all per water year) 
1. check if config file looks good, make sure to update paths with version number!!!!
1. cloud credentials updated if needed (i.e. `config/sas_token.txt` and `ee_key.json`) (note to self: use [egagli/azure_authentication](https://github.com/egagli/azure_authentication) to get new sas_token weekly)
1. `select_tiles_to_process.ipynb` ran and `tile_data/global_tiles_with_seasonal_snow.geojson` created
1. `create_zarr_store.ipynb` ran and zarr_store exists on cloud storage and can be read
1. coiled is working and using spot instances and price isn't too high--check usage stats too!!
1. check tiles are being output to `tile_data/tile_results_vX.csv` and tiles are showing success
1. check tiles are being processed with `view_maps.ipynb`, check all variables
1. validate on test tiles (check automatic weather station tile subset below)
1. check failed tiles, potentially adjust cluster settings

In [1]:
import easysnowdata
import pystac_client
import tqdm
import planetary_computer
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import odc.stac
import time
import dask
import dask.distributed
import coiled
import matplotlib.pyplot as plt
import traceback
from global_snowmelt_runoff_onset.config import Config, Tile
import global_snowmelt_runoff_onset.processing as processing
import flox

## Configuration overview

In [33]:
config = Config('../config/global_config_v9.txt')

Configuration loaded:
resolution = 0.00072000072000072
bands = vv
mountain_snow_only = False
spatial_chunk_dim_s1_read = 2048
spatial_chunk_dim_s1_process = 512
spatial_chunk_dim_zarr_output = 2048
bbox_left = -179.999
bbox_right = 179.999
bbox_top = 81.099
bbox_bottom = -59.999
wy_start = 2015
wy_end = 2024
low_backscatter_threshold = 0.001
min_monthly_acquisitions = 1
max_allowed_days_gap_per_orbit = 30
min_years_for_median_std = 3
extend_search_window_beyond_sdd_days = 16
min_consec_snow_days_for_seasonal_snow = 56
valid_tiles_geojson_path = ../processing/tile_data/global_tiles_with_seasonal_snow.geojson
tile_results_path = ../processing/tile_data/tile_results_v9.csv
global_runoff_zarr_store_azure_path = snowmelt/snowmelt_runoff_onset/global_v9.zarr
seasonal_snow_mask_zarr_store_azure_path = snowmelt/snow_cover/global_modis_snow_cover_reprojected.zarr


## Start up the coiled cluster!

In [4]:
# coiled.list_instance_types(backend="azure") <- vm types available for azure
cluster = coiled.Cluster(idle_timeout="10 minutes",
                         name="snowmelt-runoff-onset",
                         n_workers=30,
                         worker_memory="32 GB", # use 32 normally, 64 for problem tiles
                         worker_cpu=4, # use 4/8 normally, 8 for problem tiles
                         scheduler_memory="64 GB", # use 64 normally, 128 for problem tiles 
                         spot_policy="spot", # spot usually
                         environ={"GDAL_DISABLE_READDIR_ON_OPEN": "EMPTY_DIR"},
                         workspace="uwtacolab", # azure
                         )

client = cluster.get_client()
client

Output()

╭──────────────────────────────── Package Info ────────────────────────────────╮
│                                ╷                                             │
│   Package                      │ Note                                        │
│ ╶──────────────────────────────┼───────────────────────────────────────────╴ │
│   global_snowmelt_runoff_onset │ Wheel built from                            │
│                                │ ~/repos/global_snowmelt_runoff_onset        │
│                                ╵                                             │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

/home/eric/miniconda3/envs/new_global_snowmelt_runoff_onset/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


<Client: 'tls://10.0.0.108:8786' processes=19 threads=76, memory=581.40 GiB>

/home/eric/miniconda3/envs/new_global_snowmelt_runoff_onset/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/eric/miniconda3/envs/new_global_snowmelt_runoff_onset/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
# coiled.list_instance_types(backend="azure") <- vm types available for azure
cluster = coiled.Cluster(idle_timeout="10 minutes",
                         name="snowmelt-runoff-onset",
                         n_workers=60,
                         worker_memory="32 GB", # use 32 normally, 64 for problem tiles
                         worker_cpu=4, # use 4/8 normally, 8 for problem tiles
                         scheduler_memory="128 GB", # use 64 normally, 128 for problem tiles 
                         spot_policy="spot", # spot usually
                         environ={"GDAL_DISABLE_READDIR_ON_OPEN": "EMPTY_DIR"},
                         workspace="uwtacolab", # azure
                         )

client = cluster.get_client()
client

In [ ]:
# # coiled.list_instance_types(backend="azure") <- vm types available for azure
# cluster = coiled.Cluster(idle_timeout="10 minutes",
#                          name="snowmelt-runoff-onset",
#                          n_workers=60,
#                          worker_memory="16 GB", # use 32 normally, 64 for problem tiles
#                          worker_cpu=4, # use 4/8 normally, 8 for problem tiles
#                          scheduler_memory="64 GB", # use 64 normally, 128 for problem tiles 
#                          spot_policy="spot", # spot usually
#                          environ={"GDAL_DISABLE_READDIR_ON_OPEN": "EMPTY_DIR"},
#                          workspace="uwtacolab", # azure
#                          )

# client = cluster.get_client()
# client

## Tile processing function

The `process_tile` function implements the complete processing pipeline for a single spatial tile:

### Key processing steps:

1. **Sentinel-1 data retrieval**
   - Retrieve Sentinel-1 RTC data from Microsoft Planetary Computer
   - Organizes by satellite orbit and adds water year coordinates
   - Applies optimized chunking for memory management

2. **Snow cover masking** 
   - Retrieves [custom MODIS-derived seasonal snow data](https://github.com/egagli/MODIS_seasonal_snow_mask) per water year: appearance, disappearance, and maximum number of consecutive snow cover days
   - Defines pixels with seasonal snow coverage
   - Sets temporal detection windows from snow accumulation to disappearance

3. **Quality filtering**
   - Removes bad scenes and border noise artifacts
   - Filters pixels with insufficient temporal sampling
   - Calculates maximum temporal gaps per orbit

4. **Runoff onset detection**
   - Identifies minimum backscatter timing per orbit/polarization
   - Aggregates using median for robustness
   - Converts to day-of-water-year format

5. **Aggregations**
   - Computes median and MAD across water years
   - Calculates temporal resolution metrics

6. **Data output**
   - Writes results to global zarr store
   - Updates processing status tracking
   - Manages memory cleanup

In [ ]:
def process_tile(tile: Tile):
    odc.stac.configure_rio(cloud_defaults=True)
    tile.start_time = time.time()

    try:
        print(f"Getting data for tile ({tile.row},{tile.col}).")

        s1_rtc_ds = processing.get_sentinel1_rtc(
            geobox=tile.geobox,
            bands=config.bands,
            start_date=config.start_date,
            end_date=config.end_date,
            chunks_read=config.chunks_s1_read,
            fail_on_error=True,
        )

        #s1_rtc_ds["vv"] = s1_rtc_ds["vv"].chunk(config.chunks_s1_process).persist() #config.chunks_s1_process
        print("Data retrieved.")

        tile.s1_rtc_ds_dims = dict(s1_rtc_ds.sizes)

        spatiotemporal_snow_cover_mask_ds = processing.get_spatiotemporal_snow_cover_mask(
            ds=s1_rtc_ds,
            bbox_gdf=tile.bbox_gdf,
            seasonal_snow_mask_store=config.seasonal_snow_mask_store,
            extend_search_window_beyond_SDD_days=config.extend_search_window_beyond_SDD_days,
            min_consec_snow_days_for_seasonal_snow=config.min_consec_snow_days_for_seasonal_snow,
            reproject_method=config.seasonal_snow_mask_reproject_method, #precomputed for v9, otherwise rasterio
        )#.persist()

        if config.mountain_snow_only:
            gmba_clipped_gdf = processing.get_gmba_mountain_inventory(tile.bbox_gdf)
        else:
            gmba_clipped_gdf = None

        print("Applying masks...")
        s1_rtc_masked_ds = processing.apply_all_masks(
            s1_rtc_ds=s1_rtc_ds,
            gmba_clipped_gdf=gmba_clipped_gdf,
            spatiotemporal_snow_cover_mask_ds=spatiotemporal_snow_cover_mask_ds,
            water_years=config.water_years,
        )

        print("Removing bad scenes and border noise...")
        s1_rtc_masked_ds = processing.remove_bad_scenes_and_border_noise(
            s1_rtc_masked_ds, config.low_backscatter_threshold
        )
        print("Bad scenes and border noise removed.")

        print("Filtering by acquisitions and gaps...")
        s1_rtc_masked_filtered_ds = s1_rtc_masked_ds.groupby("water_year").map(
            lambda group: processing.filter_insufficient_pixels_per_orbit(
                s1_rtc_masked_ds=group,
                spatiotemporal_snow_cover_mask_ds=spatiotemporal_snow_cover_mask_ds,
                min_monthly_acquisitions=config.min_monthly_acquisitions,
                max_allowed_days_gap_per_orbit=config.max_allowed_days_gap_per_orbit,
            )
        )#.persist()

        print("Filtering completed.")


        print("Calculating temporal resolution...")
        temporal_resolution_da = processing.get_temporal_resolution(
            s1_rtc_masked_filtered_ds, spatiotemporal_snow_cover_mask_ds
        )#.persist()

        tile_median_temporal_resolution = temporal_resolution_da.median(
            dim=["latitude", "longitude"]
        )
        tile_pixel_count = temporal_resolution_da.count(dim=["latitude", "longitude"])

        tile_median_temporal_resolution, tile_pixel_count = dask.compute(
            tile_median_temporal_resolution, tile_pixel_count
        )


        for water_year in config.water_years:
            if water_year in tile_median_temporal_resolution.water_year:
                temporal_resolution = tile_median_temporal_resolution.sel(
                    water_year=water_year
                ).values
                setattr(tile, f"tr_{water_year}", round(float(temporal_resolution), 3))

            if water_year in tile_pixel_count.water_year:
                pixel_count = tile_pixel_count.sel(water_year=water_year).values
                setattr(tile, f"pix_ct_{water_year}", int(pixel_count))

        print("Temporal resolution calculated.")

        print("Calculating runoff onsets...")
        runoff_onsets_da = s1_rtc_masked_filtered_ds.groupby("water_year").apply(
            processing.calculate_runoff_onset,
            returned_dates_format="dowy",
            return_constituent_runoff_onsets=False,
        )#.persist()
        print("Runoff onsets calculated.")

        tile.runoff_onsets_dims = dict(runoff_onsets_da.sizes)

        # Calculate median and MAD
        median_da, mad_da = processing.median_and_mad_with_min_obs(
            da=runoff_onsets_da,
            dim="water_year",
            min_count=config.min_years_for_median_std
        )

        # Calculate median temporal resolution
        median_temporal_resolution_da = processing.median_with_min_obs(
            da=temporal_resolution_da,
            dim="water_year",
            min_count=config.min_years_for_median_std
        )

        # Create dataset
        runoff_onsets_ds = processing.dataarrays_to_dataset(
            runoff_onsets_da=runoff_onsets_da,
            median_da=median_da,
            mad_da=mad_da,
            water_years=config.water_years,
            temporal_resolution_da=temporal_resolution_da,
            median_temporal_resolution_da=median_temporal_resolution_da,
        )

        print("Median and MAD calculated, converted to dataset.")

        # Reindex to global coordinates
        global_ds = xr.open_zarr(config.global_runoff_store, consolidated=True)
        print("Global dataset opened.")
        global_subset_ds = global_ds.sel(
            latitude=runoff_onsets_ds.latitude,
            longitude=runoff_onsets_ds.longitude,
            method="nearest",
        )
        print("Global dataset subsetted.")
        runoff_onsets_reindexed_ds = runoff_onsets_ds.assign_coords(
            latitude=global_subset_ds.latitude, longitude=global_subset_ds.longitude
        )
        print("Dataset reindexed.")

        # Write to Zarr
        runoff_onsets_reindexed_ds.drop_vars("spatial_ref").chunk(
            config.chunks_zarr_output
        ).to_zarr(
            config.global_runoff_store, region="auto", mode="r+", consolidated=True
        )
        print("Dataset written to Zarr.")

        tile.total_time = time.time() - tile.start_time
        tile.success = True

        # Clean up memory
        del (
            s1_rtc_ds,
            spatiotemporal_snow_cover_mask_ds,
            s1_rtc_masked_ds,
            s1_rtc_masked_filtered_ds,
            temporal_resolution_da,
            runoff_onsets_da,
            runoff_onsets_ds,
            global_subset_ds,
            runoff_onsets_reindexed_ds,
            median_da,
            mad_da,
            median_temporal_resolution_da,
            tile_median_temporal_resolution,
            tile_pixel_count,
            gmba_clipped_gdf,
            global_ds,
        )

    except Exception as e:
        tile.error_messages.append(str(e))
        tile.error_messages.append(traceback.format_exc())
        tile.total_time = time.time() - tile.start_time
        tile.success = False

    return tile

## Test on a single tile

For testing, individual tiles can be processed to verify the pipeline before large-scale deployment.

In [27]:
client.restart()

In [5]:
# tiles = config.get_list_of_tiles(which='all')
# tile=tiles[0]
tile = config.get_tile(23,39)

In [ ]:
# s1_ds = processing.get_sentinel1_rtc(
#     tile.geobox,
#     bands=config.bands,
#     start_date=config.start_date,
#     end_date=config.end_date,
#     chunks_read=config.chunks_s1_read,
#     fail_on_error=True,
# )
# s1_ds

# items = (
#     pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",modifier=planetary_computer.sign_inplace)
#     .search(
#         intersects=tile.geobox.geographic_extent,
#         collections=["sentinel-1-rtc"],
#         datetime=(config.start_date, config.end_date),
#     )
#     .item_collection()
# )
# items

In [ ]:
# using to calc GB processed
# import pandas as pd
# import ast
# import numpy as np

# # Read the CSV file
# df = pd.read_csv('tile_data/tile_results_v5.csv')

# # Extract time values from s1_rtc_ds_dims column
# time_values = []
# for dims_str in df['s1_rtc_ds_dims']:
#     # Skip NaN values
#     if pd.isna(dims_str):
#         continue
#     try:
#         # Convert string representation of dict to actual dict
#         dims_dict = ast.literal_eval(dims_str)
#         time_values.append(dims_dict['time'])
#     except (ValueError, SyntaxError):
#         # Skip any malformed entries
#         continue

# # Sum all time values
# total_time_steps = sum(time_values)
# total_gb = total_time_steps * 0.0156
# print(f"Total time across all tiles: {total_time_steps}")
# print(f"Total data processed (GB): {total_gb}")
# print(f"Number of valid tiles processed: {len(time_values)}")


In [ ]:
future = client.submit(process_tile, tile)

In [ ]:
future.status

In [ ]:
future.result().success

In [ ]:
computed_result = future.result()
computed_result

In [ ]:
df = pd.DataFrame(
    [[getattr(computed_result, f) for f in config.fields]],
    columns=config.fields,
)
df
# rio 250 sec

## Tile selection and testing

Select tiles for processing based on different criteria:
- **`'all'`**: all global coverage
- **`'processed'`**: successfully completed tiles  
- **`'failed'`**: tiles that encountered errors
- **`'unprocessed'`**: tiles not yet attempted
- **`'unprocessed_and_failed'`**: tiles needing processing or reprocessing. Unless you have a specific need / debugging, you should probably use this one.
- **`'unprocessed_and_failed_weather_stations'`**: unprocessed/failed tiles that contain automatic weather stations. Useful for validation.


Provide one of these arguments to `config.get_list_of_tiles(which=)`

In [35]:
client.restart()

In [ ]:
#tiles = config.get_list_of_tiles(which='unprocessed_and_failed')
tiles = config.get_list_of_tiles(which='unprocessed_and_failed_weather_stations') # run this to process the tiles with weather stations
#tiles = config.get_list_of_tiles(which='unprocessed_and_failed')


In [ ]:
batch_size = 10
tile_batches = [tiles[i:i + batch_size] for i in range(0, len(tiles), batch_size)]

odc.stac.configure_rio(cloud_defaults=True, client=client)

for tile_batch in tqdm.tqdm(tile_batches, total=len(tile_batches)):

    futures = [client.submit(process_tile, tile) for tile in tile_batch] #, retries=0

    successful_tiles = []

    try:
        for future, computed_result in dask.distributed.as_completed(futures, with_results=True, timeout=1600):
            successful_tiles.append(computed_result)

            df = pd.DataFrame(
                [[getattr(computed_result, f) for f in config.fields]],
                columns=config.fields,
            )
            df.to_csv(config.tile_results_path, mode='a', header=False, index=False) # header=True if starting over
            print(f'Tile ({computed_result.row},{computed_result.col}) completed')
    except Exception as e:
        for tile in tile_batch:
            if tile.index not in [computed_tile.index for computed_tile in successful_tiles]:

                df = pd.DataFrame([[getattr(tile, f) for f in config.fields]],columns=config.fields,)
                df.to_csv(config.tile_results_path, mode='a', header=False, index=False) # header=True if starting over

                print(f'Tile ({tile.row},{tile.col}) failed')
                print(e)
                print(traceback.format_exc())
    
    client.restart()

  0%|          | 0/1 [00:00<?, ?it/s]

Tile (12,19) completed
Tile (13,17) completed
Tile (12,20) completed
Tile (12,21) completed
Tile (12,23) completed
Tile (23,46) completed
Tile (12,25) completed
Tile (23,47) completed
Tile (13,25) completed
Tile (23,45) completed


100%|██████████| 1/1 [14:44<00:00, 884.75s/it]


: 

In [ ]:
client.restart() # restart the client to clear memory